In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import itertools
from textblob import TextBlob
import csv
df = pd.read_csv("/home/shubhu/testing/newfinal.csv")
#ds = df.sample(frac=1)
#ds.to_csv('/home/shubhu/testing/newfinal.csv')
#print(df.head())
y = df.label 
print(y.values)
# Drop the `label` column
df = df.drop("label", axis=1)

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=53)
X_train.to_csv('/home/shubhu/testing/training.csv')
y_train.to_csv('/home/shubhu/testing/trainlabels.csv')
X_test.to_csv('/home/shubhu/testing/testing.csv')
y_test.to_csv('/home/shubhu/testing/testlabels.csv')

[1 1 0 ... 0 1 1]


In [2]:
count = 0
abc = [] #list for tfidf
all =[] #list for adding features in csv

df = pd.read_csv("/home/shubhu/testing/training.csv")

while (count < len(df.index)):
     v = df['text'].values[count].strip("b'")
     #print(v)#cleaned text
     abc.append(v)
     analysis = TextBlob(v)
     
     ratio = df['friends_count'].values[count]/df['followers_count'].values[count]
     #print(ratio)
     if df['verified'].values[count]:
        w = 1
     else:
        w = 0
        
     x = analysis.sentiment.polarity
     if x< 0:
            senti = 0
            x =x*(-1)
     elif x > 0:
            senti = 1
     else :
            senti = 2

     all.append([senti,x,analysis.sentiment.subjectivity,len(v),v.count('#'),ratio,
                 df['statuses_count'].values[count],w,v.count('@'),v.count("http"),v.count('!')])
     count = count + 1

with open('/home/shubhu/testing/trainfeatures.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["senti","polarity","subjectivity","tweet_length","No.of#","ratio","statuses_count","verified","No.of mentions",
                    "No.of links","No.of Exclamation"])
    writer.writerows(all)

    
#TFIDF feature
#tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 

# Fit and transform the training data 


    
print("hi1")
#type( df['friends_count'].values[0])       


hi1


In [9]:
count = 0
xyz = [] #list for tfidf
all =[] #list for adding features in csv

df = pd.read_csv("/home/shubhu/testing/testing.csv")

while (count < len(df.index)):
     v = df['text'].values[count].strip("b'")
     #print(v)#cleaned text
     xyz.append(v)
     analysis = TextBlob(v)
     #print(analysis.sentiment.polarity)#polarity and subjectivity feature
     #print(len(v))#length of text
     #print(v.count('#'))#No. of "#"
     ratio = df['friends_count'].values[count]/df['followers_count'].values[count]
     #print(ratio)
     if df['verified'].values[count]:
        w = 1
     else :
        w = 0
     #print(w)
     x = analysis.sentiment.polarity
     if x< 0:
            senti = 0
            x =x*(-1)
     elif x > 0:
            senti = 1
     else :
            senti = 2
     
     all.append([senti,x,analysis.sentiment.subjectivity,len(v),v.count('#'),ratio,
                 df['statuses_count'].values[count],w])
     count = count + 1

with open('/home/shubhu/testing/testfeatures.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["senti","polarity","subjectivity","tweet_length","No.of#","ratio","statuses_count","verified"])
    writer.writerows(all)
    #print(all)

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 
tfidf_train = tfidf_vectorizer.fit_transform(abc) 
# Fit and transform the training data 
tfidf_test = tfidf_vectorizer.transform(xyz) 

print("hi2")   
#type( df['followers_count'].values[0])

hi2


In [10]:

clf = MultinomialNB()

clf.fit(tfidf_train,y_train)
print("hi3")

hi3


In [11]:
pred = clf.predict(tfidf_test)
scorenb1 = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % scorenb1)



accuracy:   0.835


In [12]:
dftrain = pd.read_csv('/home/shubhu/testing/trainfeatures.csv')
dftest = pd.read_csv('/home/shubhu/testing/testfeatures.csv')

dftrain = dftrain.values # to convert df to nd array
clf = MultinomialNB()

clf.fit(dftrain,y_train)
print("hi3")

hi3


In [13]:
pred = clf.predict(dftest)
scorenb2 = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % scorenb2)


accuracy:   0.619


In [14]:
from sklearn import tree
model = tree.DecisionTreeClassifier(criterion='gini')

In [23]:
model.fit(dftrain, y_train)
#model.score(dftrain, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [24]:
predicted= model.predict(dftest)
scoredt2 = model.score(dftest,y_test)
print("The prediction accuracy is: ",model.score(dftest,y_test)*100,"%")

The prediction accuracy is:  100.0 %


In [26]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
dftrain = sc.fit_transform(dftrain)  
dftest = sc.transform(dftest)  


/home/shubhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [27]:
from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(dftrain, y_train)  
y_pred = regressor.predict(dftest)  


In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
scorerf2 = accuracy_score(y_test, y_pred)
print(accuracy_score(y_test, y_pred))  

[[20752     0]
 [    0 51092]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20752
           1       1.00      1.00      1.00     51092

   micro avg       1.00      1.00      1.00     71844
   macro avg       1.00      1.00      1.00     71844
weighted avg       1.00      1.00      1.00     71844

1.0


In [29]:
from sklearn import tree
model = tree.DecisionTreeClassifier(criterion='gini')

In [30]:
model.fit(tfidf_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [31]:
predicted= model.predict(tfidf_test)
scoredt1 = model.score(tfidf_test,y_test)
print("The prediction accuracy is: ",model.score(tfidf_test,y_test)*100,"%")

The prediction accuracy is:  84.85050943711374 %


In [32]:
from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(tfidf_train, y_train)  
y_pred = regressor.predict(tfidf_test)  


In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
scorerf1 = accuracy_score(y_test, y_pred)
print(accuracy_score(y_test, y_pred))  

[[16961  3791]
 [ 3788 47304]]
              precision    recall  f1-score   support

           0       0.82      0.82      0.82     20752
           1       0.93      0.93      0.93     51092

   micro avg       0.89      0.89      0.89     71844
   macro avg       0.87      0.87      0.87     71844
weighted avg       0.89      0.89      0.89     71844

0.8945075441233784
